**<center><font size = "5">Create custom NER model<center>**
***


In [ ]:
!pip install spacy

In [ ]:
!python3 -m spacy download en_core_web_lg

In [2]:
from __future__ import unicode_literals, print_function

import pandas as pd
import os, time

from pathlib import Path
from tqdm import tqdm 
from spacy.training import Example
from spacy import displacy
import spacy
import base64

### NER pipeline

In [3]:
nlpSpacy = spacy.load("en_core_web_lg")
 
print(nlpSpacy.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [4]:
# Load the CSV file into a DataFrame
df = pd.read_csv('known_exploited_vulnerabilities.csv')


# Create a random sample of 300 rows
sample_df = df.sample(n=300, random_state=1)

# Display the sample
print(sample_df)

                 cveID vendorProject   
919      CVE-2023-2136        Google  \
298     CVE-2021-35394       Realtek   
926      CVE-2014-0196         Linux   
323   CVE-2017-1000486      Primetek   
239     CVE-2019-16256   SIMalliance   
...                ...           ...   
322      CVE-2015-7450           IBM   
311     CVE-2021-22017        VMware   
831      CVE-2022-3236        Sophos   
1004    CVE-2023-41993         Apple   
1125    CVE-2022-24816         OSGeo   

                                                product   
919                                       Chromium Skia  \
298               Jungle Software Development Kit (SDK)   
926                                              Kernel   
323                              Primefaces Application   
239                                     Toolbox Browser   
...                                                 ...   
322   WebSphere Application Server and Server Hyperv...   
311                                      vCente

In [5]:
df.head()

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,notes,cwes
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,https://nvd.nist.gov/vuln/detail/CVE-2021-27104,"CWE-20, CWE-78"
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,https://nvd.nist.gov/vuln/detail/CVE-2021-27102,"CWE-20, CWE-78"
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,https://nvd.nist.gov/vuln/detail/CVE-2021-27101,"CWE-89, CWE-138"
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,https://nvd.nist.gov/vuln/detail/CVE-2021-27103,CWE-918
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,https://nvd.nist.gov/vuln/detail/CVE-2021-21017,CWE-122


In [6]:
text1 = sample_df["vulnerabilityName"]
text1.head()


919    Google Chrome Skia Integer Overflow Vulnerability
298    Realtek Jungle SDK Remote Code Execution Vulne...
926            Linux Kernel Race Condition Vulnerability
323    Primetek Primefaces Remote Code Execution Vuln...
239    SIMalliance Toolbox Browser Command Injection ...
Name: vulnerabilityName, dtype: object

In [7]:
text2 = sample_df["shortDescription"]
text2.head()

919    Google Chromium Skia contains an integer overf...
298    RealTek Jungle SDK contains multiple memory co...
926    Linux Kernel contains a race condition vulnera...
323    Primetek Primefaces is vulnerable to a weak en...
239    SIMalliance Toolbox Browser contains an comman...
Name: shortDescription, dtype: object

In [10]:
# Save text1 to a file
with open('text1.txt', 'w') as file1:
    file1.write('\n'.join(text1))

# Save text2 to a file
with open('text2.txt', 'w') as file2:
    file2.write('\n'.join(text2))

In [13]:
file = open('text2.txt')
text = file.read()
file.close()
doc = nlpSpacy(text)
displacy.render(doc, style="ent", jupyter=True)

In [14]:
for entity in doc.ents:
  print(f"Name of the entity {entity.text} type is {entity.label_}")

Name of the entity Google Chromium Skia type is ORG
Name of the entity Google Chrome type is ORG
Name of the entity RealTek Jungle type is PERSON
Name of the entity Linux Kernel type is PRODUCT
Name of the entity Primetek Primefaces type is ORG
Name of the entity SIMalliance Toolbox Browser type is PERSON
Name of the entity Microsoft Windows Adobe Font Manager Library type is ORG
Name of the entity Adobe Type type is PRODUCT
Name of the entity 1 type is CARDINAL
Name of the entity Windows 10 type is PRODUCT
Name of the entity Windows 10 type is PRODUCT
Name of the entity AppContainer type is ORG
Name of the entity Google Chrome WebAudio type is PRODUCT
Name of the entity Microsoft Internet Explorer type is ORG
Name of the entity 2 type is CARDINAL
Name of the entity Microsoft Internet Explorer type is ORG
Name of the entity Apache CouchDB type is PRODUCT
Name of the entity Apple iOS type is ORG
Name of the entity TOCTOU type is ORG
Name of the entity Pointer Authentication type is FAC


## Annotate the text for NER 

Navigate to 

https://tecoholic.github.io/ner-annotator/

and make all the custom NER and annotate the entities on the text one by one

## Make a Spacy custom NER MODEL

In [15]:
import json
 
with open('annotations 1.json', 'r') as f:
    data = json.load(f)
    
print(data['annotations'][0])

['Google Chrome Skia Integer Overflow Vulnerability', {'entities': [[0, 6, 'ORG'], [7, 13, 'PRODUCT'], [14, 18, 'LIBRARY'], [19, 35, 'FIELD']]}]


In [16]:
data['classes']

['PRODUCT', 'FIELD', 'LIBRARY', 'ORG', 'FUNCTION']

## Create empty model

In [17]:
training_data = data.copy()
training_data['classes'] = data['classes']
training_data['annotations'] = []
for text, annotation in data['annotations']:
    try:
        if text!="" and len(annotation['entities']) > 0:
            temp_dict = {}
            temp_dict['text'] = text
            temp_dict['entities'] = []
            for ent in annotation['entities']:
                start = ent[0]
                end = ent[1]
                label = ent[2].upper()
                temp_dict['entities'].append((start, end, label))
        training_data['annotations'].append(temp_dict)
    except:
        pass
print(training_data)

{'classes': ['PRODUCT', 'FIELD', 'LIBRARY', 'ORG', 'FUNCTION'], 'annotations': [{'text': 'Google Chrome Skia Integer Overflow Vulnerability', 'entities': [(0, 6, 'ORG'), (7, 13, 'PRODUCT'), (14, 18, 'LIBRARY'), (19, 35, 'FIELD')]}, {'text': 'Realtek Jungle SDK Remote Code Execution Vulnerability', 'entities': [(0, 7, 'ORG'), (8, 18, 'PRODUCT'), (19, 40, 'FIELD')]}, {'text': 'Linux Kernel Race Condition Vulnerability', 'entities': [(0, 5, 'ORG'), (6, 27, 'FIELD')]}, {'text': 'Primetek Primefaces Remote Code Execution Vulnerability', 'entities': [(0, 8, 'ORG'), (9, 19, 'PRODUCT'), (20, 41, 'FIELD')]}, {'text': 'SIMalliance Toolbox Browser Command Injection Vulnerability', 'entities': [(0, 11, 'ORG'), (12, 27, 'PRODUCT'), (28, 45, 'FIELD')]}, {'text': 'Microsoft Windows Adobe Font Manager Library Remote Code Execution Vulnerability', 'entities': [(0, 9, 'ORG'), (10, 17, 'PRODUCT'), (18, 36, 'FUNCTION'), (45, 66, 'FIELD')]}, {'text': 'Google Chrome WebAudio Use-After-Free Vulnerability', '

### Configuration variables

In [18]:
modelSpacy = None
n_iter=100

### Load the model

In [19]:
if modelSpacy is not None:
    nlp = spacy.load(modelSpacy)  
    print("Loaded model '%s'" % modelSpacy)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


### Set up pipeline

In [20]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

### Train the NER model

In [21]:
#getting all the entities
for annotations in training_data["annotations"]:
    for ent in annotations['entities']:
        ner.add_label(ent[2])
        
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        losses = {}
        for text, annotations in tqdm(data["annotations"]):
            try:
                if text!="":
                    example = Example.from_dict(nlp.make_doc(text), annotations)
                    #Update the model
                    nlp.update(
                        [example],  
                        drop=0.5,  
                        sgd=optimizer,
                        losses=losses)
            except:
                pass
        print(losses)


100%|██████████| 300/300 [00:02<00:00, 129.48it/s]


{'ner': 1281.9669577487327}


100%|██████████| 300/300 [00:01<00:00, 150.50it/s]


{'ner': 840.0535356115821}


100%|██████████| 300/300 [00:02<00:00, 144.71it/s]


...

{'ner': 71.09343940391909}


100%|██████████| 300/300 [00:01<00:00, 152.87it/s]


{'ner': 77.6210527065922}


100%|██████████| 300/300 [00:01<00:00, 152.72it/s]

{'ner': 75.89528029367014}


### Test the trained model

In [22]:
for annotations in training_data["annotations"]:
    doc = nlp(annotations['text'])
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Google', 'ORG'), ('Chrome', 'PRODUCT'), ('Skia', 'LIBRARY'), ('Integer Overflow', 'FIELD')]
Tokens [('Google', 'ORG', 3), ('Chrome', 'PRODUCT', 3), ('Skia', 'LIBRARY', 3), ('Integer', 'FIELD', 3), ('Overflow', 'FIELD', 1), ('Vulnerability', '', 2)]
Entities [('Realtek', 'ORG'), ('Jungle SDK', 'PRODUCT'), ('Remote Code Execution', 'FIELD')]
Tokens [('Realtek', 'ORG', 3), ('Jungle', 'PRODUCT', 3), ('SDK', 'PRODUCT', 1), ('Remote', 'FIELD', 3), ('Code', 'FIELD', 1), ('Execution', 'FIELD', 1), ('Vulnerability', '', 2)]
Entities [('Linux Kernel Race', 'PRODUCT')]
Tokens [('Linux', 'PRODUCT', 3), ('Kernel', 'PRODUCT', 1), ('Race', 'PRODUCT', 1), ('Condition', '', 2), ('Vulnerability', '', 2)]
Entities [('Primetek', 'ORG'), ('Primefaces', 'PRODUCT'), ('Remote Code Execution', 'FIELD')]
Tokens [('Primetek', 'ORG', 3), ('Primefaces', 'PRODUCT', 3), ('Remote', 'FIELD', 3), ('Code', 'FIELD', 1), ('Execution', 'FIELD', 1), ('Vulnerability', '', 2)]
Entities [('SIMalliance', 'ORG'), ('T

### Save the cusom NER model

In [23]:
output_dir= os.path.abspath(os.getcwd())
nlp.to_disk(output_dir)
print("Saved model to", output_dir) 

Saved model to /Users/dj/Desktop/Capstone/Capstone


### Test the saved custom model

In [24]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for annotations in training_data["annotations"][3:5]:
    doc = nlp2(annotations['text'])
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from /Users/dj/Desktop/Capstone/Capstone
Entities [('Primetek', 'ORG'), ('Primefaces', 'PRODUCT'), ('Remote Code Execution', 'FIELD')]
Tokens [('Primetek', 'ORG', 3), ('Primefaces', 'PRODUCT', 3), ('Remote', 'FIELD', 3), ('Code', 'FIELD', 1), ('Execution', 'FIELD', 1), ('Vulnerability', '', 2)]
Entities [('SIMalliance', 'ORG'), ('Toolbox Browser', 'PRODUCT'), ('Command Injection', 'FIELD')]
Tokens [('SIMalliance', 'ORG', 3), ('Toolbox', 'PRODUCT', 3), ('Browser', 'PRODUCT', 1), ('Command', 'FIELD', 3), ('Injection', 'FIELD', 1), ('Vulnerability', '', 2)]


In [25]:
file = open('text2.txt')
text = file.read()
file.close()
nlp2 = spacy.load(output_dir)
doc = nlp2(text)
displacy.render(doc, style="ent", jupyter=True)

In [26]:
import json
 
with open('annotations.json', 'r') as f:
    data = json.load(f)
    
print(data['annotations'][0])
data['classes']
## Create empty model
training_data = data.copy()
training_data['classes'] = data['classes']
training_data['annotations'] = []
for text, annotation in data['annotations']:
    try:
        if text!="" and len(annotation['entities']) > 0:
            temp_dict = {}
            temp_dict['text'] = text
            temp_dict['entities'] = []
            for ent in annotation['entities']:
                start = ent[0]
                end = ent[1]
                label = ent[2].upper()
                temp_dict['entities'].append((start, end, label))
        training_data['annotations'].append(temp_dict)
    except:
        pass
print(training_data)
### Configuration variables
modelSpacy = None
n_iter=100
### Load the model
if modelSpacy is not None:
    nlp = spacy.load(modelSpacy)  
    print("Loaded model '%s'" % modelSpacy)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")
### Set up pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')
### Train the NER model
#getting all the entities
for annotations in training_data["annotations"]:
    for ent in annotations['entities']:
        ner.add_label(ent[2])
        
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        losses = {}
        for text, annotations in tqdm(data["annotations"]):
            try:
                if text!="":
                    example = Example.from_dict(nlp.make_doc(text), annotations)
                    #Update the model
                    nlp.update(
                        [example],  
                        drop=0.5,  
                        sgd=optimizer,
                        losses=losses)
            except:
                pass
        print(losses)

### Test the trained model
for annotations in training_data["annotations"]:
    doc = nlp(annotations['text'])
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
### Save the cusom NER model
output_dir= os.path.abspath(os.getcwd())
nlp.to_disk(output_dir)
print("Saved model to", output_dir) 
### Test the saved custom model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for annotations in training_data["annotations"][3:5]:
    doc = nlp2(annotations['text'])
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
file = open('text2.txt')
text = file.read()
file.close()
nlp2 = spacy.load(output_dir)
doc = nlp2(text)
displacy.render(doc, style="ent", jupyter=True)

['Microsoft Office Publisher contains a security feature bypass vulnerability that allows for a local, authenticated attack on a targeted system.', {'entities': [[0, 9, 'VENDOR'], [10, 26, 'PRODUCT'], [38, 61, 'VULNERABILITY NAME'], [94, 121, 'ACTOR'], [127, 142, 'MODULE']]}]
{'classes': ['VENDOR', 'PRODUCT', 'FEATURE', 'ACTOR', 'EXPLOIT', 'MODULE', 'VULNERABILITY NAME'], 'annotations': [{'text': 'Microsoft Office Publisher contains a security feature bypass vulnerability that allows for a local, authenticated attack on a targeted system.', 'entities': [(0, 9, 'VENDOR'), (10, 26, 'PRODUCT'), (38, 61, 'VULNERABILITY NAME'), (94, 121, 'ACTOR'), (127, 142, 'MODULE')]}, {'text': 'Microsoft Exchange Server contains an unspecified vulnerability that allows for remote code execution. This vulnerability is part of the ProxyLogon exploit chain.', 'entities': [(0, 9, 'VENDOR'), (10, 25, 'PRODUCT'), (38, 49, 'VULNERABILITY NAME'), (80, 102, 'EXPLOIT'), (137, 147, 'EXPLOIT')]}, {'text': 'Unspecifi

100%|██████████| 31/31 [00:00<00:00, 80.35it/s]


{'ner': 511.01224369685497}


100%|██████████| 31/31 [00:00<00:00, 72.73it/s]


{'ner': 324.8892041780855}


...

{'ner': 48.065608580371034}


100%|██████████| 31/31 [00:00<00:00, 76.80it/s]


{'ner': 19.537979260562864}


100%|██████████| 31/31 [00:00<00:00, 69.34it/s]


{'ner': 16.280409569208356}
Entities [('Microsoft', 'VENDOR'), ('Office Publisher', 'PRODUCT'), ('security feature bypass', 'VULNERABILITY NAME'), ('local, authenticated attack', 'ACTOR'), ('targeted system', 'MODULE')]
Tokens [('Microsoft', 'VENDOR', 3), ('Office', 'PRODUCT', 3), ('Publisher', 'PRODUCT', 1), ('contains', '', 2), ('a', '', 2), ('security', 'VULNERABILITY NAME', 3), ('feature', 'VULNERABILITY NAME', 1), ('bypass', 'VULNERABILITY NAME', 1), ('vulnerability', '', 2), ('that', '', 2), ('allows', '', 2), ('for', '', 2), ('a', '', 2), ('local', 'ACTOR', 3), (',', 'ACTOR', 1), ('authenticated', 'ACTOR', 1), ('attack', 'ACTOR', 1), ('on', '', 2), ('a', '', 2), ('targeted', 'MODULE', 3), ('system', 'MODULE', 1), ('.', '', 2)]
Entities [('Microsoft', 'VENDOR'), ('Exchange Server', 'PRODUCT'), ('unspecified', 'VULNERABILITY NAME'), ('remote code execution.', 'EXPLOIT'), ('ProxyLogon', 'EXPLOIT')]
Tokens [('Microsoft', 'VENDOR', 3), ('Exchange', 'PRODUCT', 3), ('Server', 'PRODUCT'